In [1]:
%gui qt
import sys
from pathlib import Path
import urllib3

from dataclasses import dataclass
from scipy.spatial import ConvexHull

import numpy as np

import meshio
import zarr
import napari

sys.path.append("..")
import ply_zarr

from ome_zarr.utils import download

In [2]:
idr_data = "https://s3.embassy.ebi.ac.uk/idr/zarr/v0.1/6001247.zarr/"


In [3]:
idr_name = idr_data.rstrip('/').split("/")[-1]

In [4]:
local_path = Path("../data/") / idr_name
if not local_path.exists():
    download(idr_data, local_path.as_posix())


In [5]:
data = zarr.open(local_path.as_posix(), "a")

In [6]:
data.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, icon='table', name='0 (1, 2, 257, 210, 25…

In [7]:
labels = data['labels']["masks"][0][0]

In [8]:
lbls = np.unique(labels)

In [9]:
arr = data[0]

In [10]:
meshes = []
for idx in lbls[1:6]:
    _, x, y, z = (labels == idx).nonzero()
    points = np.vstack([x, y, z]).T
    hull = ConvexHull(points)
    triangles = hull.simplices
    color = (points * [0.1, 0.3, 0.2]).sum(axis=1)
    
    mesh = meshio.Mesh(
        points=points,
        cells=[("triangle", triangles),],
        point_data={"color": color})
    
    mesh.remove_orphaned_nodes()
    
    meshes.append(mesh)

In [11]:
viewer = napari.Viewer()
viewer.add_image(arr)

for mesh in meshes:
    viewer.add_surface(
        (mesh.points,
         mesh.cells[0].data,
         mesh.point_data["color"]),
        colormap="Blues", opacity=1
    )

In [12]:
!rm -Rf {local_path / "meshes"}

In [13]:
mesh_grp = data.create_group('meshes')

In [14]:
for i, mesh in enumerate(meshes):
    grp = mesh_grp.create_group(i)
    ply_zarr.write(grp, mesh)


In [15]:
data.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, icon='table', name='0 (1, 2, 257, 210, 25…

In [16]:
print(mesh_grp[0].tree())

0
 ├── 3
 │   └── vertex_indices (336, 3) int32
 └── points
     ├── color (170,) float64
     ├── x (170,) int64
     ├── y (170,) int64
     └── z (170,) int64


In [23]:
mesh_reloaded = ply_zarr.read(mesh_grp[0])

KeyError: 'vertex_indices'

In [22]:

mesh_grp[0].attrs.asdict()

{'comments': ['Created by ply-zarr v0.0.1, 2021-05-05T16:39:44.104339'],
 'elements': {'face': {'properties': [['list',
     'uint8',
     'int32',
     'vertex_indices']],
   'size': 336},
  'vertex': {'properties': [['int64', 'x'],
    ['int64', 'y'],
    ['int64', 'z'],
    ['double', 'color']],
   'size': 170}},
 'format': 'ascii 1.0'}